In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

In [2]:
os.getcwd()

'C:\\Users\\IN0202'

In [3]:
os.chdir(r"D:\Personal\Analytics course\IPBA\Case Study\hundred")
path=r"D:\Personal\Analytics course\IPBA\Case Study\hundred"

In [4]:
os.getcwd()

'D:\\Personal\\Analytics course\\IPBA\\Case Study\\hundred'

In [5]:
from os import listdir
filepaths = [f for f in listdir(path) if f.startswith('Stock')]
print(filepaths)

['Stock_ABBOTINDIA.xlsx', 'Stock_ACC.xlsx', 'Stock_ADANIPORTS.xlsx', 'Stock_ADANITRANS.xlsx', 'Stock_AMBUJACEM.xlsx', 'Stock_ASIANPAINT.xlsx', 'Stock_AUROPHARMA.xlsx', 'Stock_AXISBANK.xlsx', 'Stock_BAJAJ-AUTO.xlsx', 'Stock_BAJAJFINSV.xlsx', 'Stock_BAJAJHLDNG.xlsx', 'Stock_BAJFINANCE.xlsx', 'Stock_BANDHANBNK.xlsx', 'Stock_BANKBARODA.xlsx', 'Stock_BERGEPAINT.xlsx', 'Stock_BHARTIARTL.xlsx', 'Stock_BIOCON.xlsx', 'Stock_BOSCHLTD.xlsx', 'Stock_BPCL.xlsx', 'Stock_BRITANNIA.xlsx', 'Stock_CADILAHC.xlsx', 'Stock_CIPLA.xlsx', 'Stock_COALINDIA.xlsx', 'Stock_COLPAL.xlsx', 'Stock_CONCOR.xlsx', 'Stock_DABUR.xlsx', 'Stock_DIVISLAB.xlsx', 'Stock_DLF.xlsx', 'Stock_DMART.xlsx', 'Stock_DRREDDY.xlsx', 'Stock_EICHERMOT.xlsx', 'Stock_GAIL.xlsx', 'Stock_GICRE.xlsx', 'Stock_GODREJCP.xlsx', 'Stock_GRASIM.xlsx', 'Stock_HAVELLS.xlsx', 'Stock_HCLTECH.xlsx', 'Stock_HDFC.xlsx', 'Stock_HDFCAMC.xlsx', 'Stock_HDFCBANK.xlsx', 'Stock_HDFCLIFE.xlsx', 'Stock_HEROMOTOCO.xlsx', 'Stock_HINDALCO.xlsx', 'Stock_HINDPETRO.xlsx', 

In [6]:
# Read the data from each excel and store all of them into raw_data
daily_data= pd.concat(map(pd.read_excel, filepaths))

In [ ]:
#daily_data.head(10)
daily_data.tail(10)

In [7]:
daily_data.shape

(244901, 15)

In [8]:
daily_data.isnull().sum()

Date                      0
Symbol                    0
Series                    0
Prev Close                0
Open                      0
High                      0
Low                       0
Last                      0
Close                     0
VWAP                      0
Volume                    0
Turnover                  0
Trades                30684
Deliverable Volume        0
%Deliverble               0
dtype: int64

In [9]:
# while doing OBV realized that Adaniport has Mundraport initially and then symbol changed to adaniport, 
# hence determined the Number of unique Symbol - to realize there are 6 such cases
daily_data['Symbol'].nunique()
#on further analysis of the concatenated files (using excel) - see that its for \n",
# (a) MUNDRAPORT & ADANIPORTS,
# (b) BAJAUTOFIN & BAJFINANCE,
# (c) HEROHONDA & HEROMOTOCO,
# (d) PIRHEALTH & PEL,
# (e) UNIPHOS & UPL,
# went ahead and manually changed in the excel and re-read the data

106

In [10]:
# Delete and reload the data frame with corrected data
del daily_data

In [11]:
#reload the corrected SYbmbols
os.chdir(r"D:\Personal\Analytics course\IPBA\Case Study\hundred_corrected")
path=r"D:\Personal\Analytics course\IPBA\Case Study\hundred_corrected"

# read the data from the files
daily_data= pd.concat(map(pd.read_excel, filepaths))

In [12]:
# check now if the number of stocks match to 100
daily_data['Symbol'].nunique()

100

In [13]:
# to calculate the simple moving average - 5, 10 , 50, 100, 200 days
daily_data['SMA-5'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=5).mean())
daily_data['SMA-10'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=10).mean())
daily_data['SMA-50'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=50).mean())
daily_data['SMA-100'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=100).mean())
daily_data['SMA-200'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=200).mean())

In [14]:
# to calculate the exponential moving average - 5, 10 , 50, 100, 200 days
daily_data['EMA-5'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=5).mean())
daily_data['EMA-10'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=10).mean())
daily_data['EMA-50'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=50).mean())
daily_data['EMA-100'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=100).mean())
daily_data['EMA-200'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=200).mean())

In [15]:
# calculcate RSI - Relative strength index
change = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.diff())
gain = change.mask(change < 0, 0)
loss = change.mask (change > 0, 0)
average_gain= gain.rolling(window = 14).mean().shift()
average_loss= loss.rolling(window = 14).mean().shift()
relativestrength = abs(average_gain / average_loss)
rsi = 100 - (100/(1+relativestrength))
daily_data['rsi'] = rsi.shift(-1) # move up since the current column is already on next row due to rolling function

In [16]:
# calculate MACD
# MACD = 26-day EMA - 12-day EMA
daily_data['ema12'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=12).mean())
daily_data['ema26'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.ewm(span=26).mean())
daily_data['macd'] = daily_data['ema12'] - daily_data['ema26']
## for cross over - calculate macd9
daily_data['macd9'] = daily_data.groupby('Symbol')['macd'].transform(lambda x:x.ewm(span=9).mean())

In [17]:
# calculate bollinger band
daily_data['SMA-20'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=20).mean())
daily_data['stddev-20'] = daily_data.groupby('Symbol')['Close'].transform(lambda x:x.rolling(window=20).std())
daily_data['Bollinger-UpperBand'] = daily_data['SMA-20'] + 2*daily_data['stddev-20']
daily_data['Bollinger-LowerBand'] = daily_data['SMA-20'] - 2*daily_data['stddev-20']

In [ ]:
# calculate the OBV
#daily_data.groupby('Symbol')['OBV'] = np.where(daily_data.groupby('Symbol')['Close'] > daily_data.groupby('Symbol')['Close'].shift(1), daily_data.groupby('Symbol')['Volume'], \
#                                               np.where(daily_data.groupby('Symbol')['Close'] < daily_data.groupby('Symbol')['Close'].shift(1), - daily_data.groupby('Symbol')['Volume'], 0)).cumsum()

In [ ]:
# user defined function
#def on_balance_volume(temp_data, n):
#    i = 0
#    OBV = [0]
#    for index, row in temp_data.iterrows():
#        while i < temp_data.index[-1]:
#            if temp_data.loc[i + 1, 'Close'] - temp_data.loc[i, 'Close'] > 0
#                OBV.append(temp_data.loc[i + 1, 'Volume'])
#            if temp_data.loc[i + 1, 'Close'] - temp_data.loc[i, 'Close'] == 0
#                OBV.append(0)
#            if temp_data.loc[i + 1, 'Close'] - temp_data.loc[i, 'Close'] < 0:\n",
#                OBV.append(-temp_data.loc[i + 1, 'Volume'])
#        i = i + 1
#    OBV = pd.Series(OBV)
#    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean(), name='OBV_' + str(n))
#    temp_data = temp_data.join(OBV_ma)
#return temp_data

In [18]:
# Calculate the OBV - based on price change, denote whether the volume needs to be added or subtracted in a new column - store it as  list in OBV
df1_grouped = daily_data.groupby('Symbol')
OBV = [0]
for group_keys, df_group in df1_grouped:
    i = 0
    for row_index, row in df_group.iterrows():
        if i <= df_group.index[-1] :
            if i == 0 :
                OBV.append(df_group.loc[row_index, 'Volume'])
            else :
                if df_group.loc[row_index, 'Close'] > df_group.loc[row_index - 1, 'Close'] :
                    OBV.append(df_group.loc[row_index, 'Volume'])
                if df_group.loc[row_index, 'Close'] == df_group.loc[row_index - 1, 'Close'] :
                    OBV.append(0)
                if df_group.loc[row_index, 'Close'] < df_group.loc[row_index - 1, 'Close'] :
                    OBV.append(-df_group.loc[row_index, 'Volume'])
        i = i + 1

In [19]:
OBV

[0,
 51,
 -897,
 -426,
 -359,
 317,
 6965,
 140,
 -2420,
 -527,
 -980,
 0,
 -818,
 -558,
 125,
 -132,
 792,
 1572,
 1095,
 -325,
 -390,
 -124,
 344,
 498,
 -794,
 1051,
 -1790,
 2148,
 1737,
 38206,
 -11640,
 3271,
 11868,
 8727,
 -5490,
 -1890,
 -6883,
 -8592,
 6626,
 -592,
 2000,
 -415,
 -1341,
 -532,
 669,
 -887,
 13443,
 1858,
 -541,
 632,
 4170,
 2166,
 -2295,
 1312,
 7531,
 -2353,
 -8511,
 1583,
 1146,
 -1740,
 1218,
 -733,
 -1676,
 1202,
 -1621,
 2965,
 -10434,
 1753,
 3174,
 11535,
 5089,
 1377,
 -1880,
 -315,
 -309,
 2013,
 -212,
 391,
 -3090,
 1766,
 1788,
 4160,
 -6644,
 104,
 2337,
 309,
 8513,
 3529,
 25341,
 -4601,
 -1550,
 414100,
 -45171,
 -63400,
 19015,
 -8698,
 -13945,
 -7192,
 9929,
 -9244,
 2432,
 10236,
 8649,
 87690,
 25348,
 -7809,
 -5610,
 -5914,
 17990,
 -7372,
 -3506,
 -3177,
 -15853,
 3360,
 4360,
 -2621,
 -5076,
 3474,
 29543,
 -9874,
 -8111,
 8072,
 8950,
 24248,
 -15976,
 6810,
 4827,
 5030,
 -3682,
 -2585,
 -2139,
 -2743,
 4275,
 4245,
 -1351,
 -1914,
 -

In [20]:
# convert the series to dataframe with column as OBV
dataframe_OBV = pd.DataFrame(OBV,columns=['OBV'])
dataframe_OBV

,OBV
0,0
1,51
2,-897
3,-426
4,-359
...,...
244897,61557172
244898,34075363
244899,-45045743
244900,31203625


In [21]:
# remove the first row since the append was started with OBV[0] - this way this would align to the length of the original dataframe
dataframe_OBV_corrected = dataframe_OBV[1:]
dataframe_OBV_corrected.tail()

,OBV
244897,61557172
244898,34075363
244899,-45045743
244900,31203625
244901,19070023


In [22]:
# add the non-cumulative OBV value to the existing daily_data dataframe
daily_data['OBV Non-Cum'] = dataframe_OBV_corrected['OBV'].values

In [23]:
daily_data.head(2670)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,...,rsi,ema12,ema26,macd,macd9,SMA-20,stddev-20,Bollinger-UpperBand,Bollinger-LowerBand,OBV Non-Cum
0,2010-01-08,ABBOTINDIA,EQ,783.75,755.00,807.0,755.00,807.00,807.00,784.12,...,NaN,807.000000,807.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,51
1,2010-01-11,ABBOTINDIA,EQ,807.00,825.00,825.0,790.00,804.80,804.80,793.90,...,NaN,805.808333,805.857692,-0.049359,-0.027422,NaN,NaN,NaN,NaN,-897
2,2010-01-12,ABBOTINDIA,EQ,804.80,800.00,805.0,790.00,790.00,790.00,799.36,...,NaN,799.638337,800.160177,-0.521840,-0.230052,NaN,NaN,NaN,NaN,-426
3,2010-01-13,ABBOTINDIA,EQ,790.00,787.00,787.0,720.00,767.00,767.00,758.53,...,NaN,789.335690,790.890041,-1.554351,-0.678663,NaN,NaN,NaN,NaN,-359
4,2010-01-14,ABBOTINDIA,EQ,767.00,769.00,798.0,769.00,785.10,785.10,791.73,...,NaN,788.184868,789.547307,-1.362439,-0.882071,NaN,NaN,NaN,NaN,317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,2020-09-30,ABBOTINDIA,EQ,16402.00,16365.00,16500.0,16235.35,16498.00,16486.95,16444.31,...,49.279947,16328.398617,16376.119591,-47.720974,-39.347659,16415.2975,282.364335,16980.026169,15850.568831,6371
2666,2020-10-01,ABBOTINDIA,EQ,16486.95,16470.00,16595.0,16325.00,16395.05,16384.90,16420.26,...,41.407347,16337.091138,16376.769992,-39.678854,-39.413898,16412.3175,282.354420,16977.026339,15847.608661,-6133
0,2010-01-04,ACC,EQ,872.45,870.00,920.0,870.00,919.10,913.60,898.90,...,NaN,913.600000,913.600000,0.000000,0.000000,NaN,NaN,NaN,NaN,852434
1,2010-01-05,ACC,EQ,913.60,920.55,926.4,898.05,901.00,901.75,913.48,...,NaN,907.181250,907.447115,-0.265865,-0.147703,NaN,NaN,NaN,NaN,-439209


In [24]:
# do the cumulative sum based on the grouping iof each symbol to get the OBV for each Symbol
daily_data['OBV'] = daily_data.groupby('Symbol')['OBV Non-Cum'].cumsum()

In [25]:
daily_data.head(2670)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,...,ema12,ema26,macd,macd9,SMA-20,stddev-20,Bollinger-UpperBand,Bollinger-LowerBand,OBV Non-Cum,OBV
0,2010-01-08,ABBOTINDIA,EQ,783.75,755.00,807.0,755.00,807.00,807.00,784.12,...,807.000000,807.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,51,51
1,2010-01-11,ABBOTINDIA,EQ,807.00,825.00,825.0,790.00,804.80,804.80,793.90,...,805.808333,805.857692,-0.049359,-0.027422,NaN,NaN,NaN,NaN,-897,-846
2,2010-01-12,ABBOTINDIA,EQ,804.80,800.00,805.0,790.00,790.00,790.00,799.36,...,799.638337,800.160177,-0.521840,-0.230052,NaN,NaN,NaN,NaN,-426,-1272
3,2010-01-13,ABBOTINDIA,EQ,790.00,787.00,787.0,720.00,767.00,767.00,758.53,...,789.335690,790.890041,-1.554351,-0.678663,NaN,NaN,NaN,NaN,-359,-1631
4,2010-01-14,ABBOTINDIA,EQ,767.00,769.00,798.0,769.00,785.10,785.10,791.73,...,788.184868,789.547307,-1.362439,-0.882071,NaN,NaN,NaN,NaN,317,-1314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,2020-09-30,ABBOTINDIA,EQ,16402.00,16365.00,16500.0,16235.35,16498.00,16486.95,16444.31,...,16328.398617,16376.119591,-47.720974,-39.347659,16415.2975,282.364335,16980.026169,15850.568831,6371,2547264
2666,2020-10-01,ABBOTINDIA,EQ,16486.95,16470.00,16595.0,16325.00,16395.05,16384.90,16420.26,...,16337.091138,16376.769992,-39.678854,-39.413898,16412.3175,282.354420,16977.026339,15847.608661,-6133,2541131
0,2010-01-04,ACC,EQ,872.45,870.00,920.0,870.00,919.10,913.60,898.90,...,913.600000,913.600000,0.000000,0.000000,NaN,NaN,NaN,NaN,852434,852434
1,2010-01-05,ACC,EQ,913.60,920.55,926.4,898.05,901.00,901.75,913.48,...,907.181250,907.447115,-0.265865,-0.147703,NaN,NaN,NaN,NaN,-439209,413225


In [44]:
daily_data.isnull().sum()

Date                       0
Symbol                     0
Series                     0
Prev Close                 0
Open                       0
High                       0
Low                        0
Last                       0
Close                      0
VWAP                       0
Volume                     0
Turnover                   0
Trades                 30684
Deliverable Volume         0
%Deliverble                0
SMA-5                    400
SMA-10                   900
SMA-50                  4900
SMA-100                 9900
SMA-200                19839
EMA-5                      0
EMA-10                     0
EMA-50                     0
EMA-100                    0
EMA-200                    0
rsi                     1401
ema12                      0
ema26                      0
macd                       0
macd9                      0
SMA-20                  1900
stddev-20               1900
Bollinger-UpperBand     1900
Bollinger-LowerBand     1900
OBV Non-Cum   

In [45]:
# determine Bearish or Bullish market and the trade strategy based on macd and macd9
# if macd > macd9, "bullish" and "buy"
# if macd < macd9, "bearish" and "sell"
daily_data['TradingStrategymacd'] = np.where(daily_data['macd'] > daily_data['macd9'], "Buy", "Sell")

In [46]:
# determine the trading strategy for Bollinger
# Buy - first trade confirmation we need is for the price to break and close above the middle Bollinger band
# Sell - Close price drops from a Buy region of y/day to below the middle Bollinger band
daily_data['TradingStrategyBollinger'] = np.where(daily_data['Close'] >= daily_data['SMA-20'], "Buy", "Sell")

In [47]:
# determine the trading strategy for RSI
# RSI reading above the 50 level is considered as a positive momentum 
# RSI reading below the 50 level is considered negative momentum
daily_data['TradingStrategyRSI'] = np.where(daily_data['rsi'] > 50, "positive", "negative")

In [48]:
# determine the trading strategy for OBV
# Buy at the market once you see volume confirming the price
daily_data['TradingStrategyOBV'] = np.where((daily_data['OBV'] > daily_data['OBV'].shift(1)) & (daily_data['OBV'] > 0), "Buy", "Sell")

In [62]:
# HIDE THE PROTECTIVE STOP LOSS BELOW THE BOLLINGER BAND
# The logical place to hide your protective stop loss is below the lower Bollinger band. 
# A break below the lower BB will invalidate our trade idea, and we want to minimize our losses.

# STRATEGY FOR EXIT FROM THE MARKET (MAKE A SELL DECISION)
# A break below the lower Bollinger Band is a good signal for a possible reversal, so we want to cash out our profits
daily_data['STOPLOSS'] = np.where((daily_data['Close'] < daily_data['Bollinger-LowerBand']), "STOP", "WAIT")

In [68]:
# COMBINED STRATEGY TO ENTER THE MARKET (MAKE A BUY DECISION)
# Step #1: Price needs to Break and Close above the middle Bollinger Band
# Step #2: Wait for the RSI indicator to trade above the 50 level if it doesn’t already
# Step #3: Wait for the OBV indicator to rise. Buy at the market once you see volume confirming the price

# HIDE THE PROTECTIVE STOP LOSS BELOW THE BOLLINGER BAND
# The logical place to hide your protective stop loss is below the lower Bollinger band. 
# A break below the lower BB will invalidate our trade idea, and we want to minimize our losses.

# STRATEGY FOR EXIT FROM THE MARKET (MAKE A SELL DECISION)
# A break below the lower Bollinger Band is a good signal for a possible reversal, so we want to cash out our profits

df1_grouped = daily_data.groupby('Symbol')
COMBINED = [0]
for group_keys, df_group in df1_grouped:
    i = 0
    for row_index, row in df_group.iterrows():
        if i <= df_group.index[-1] :
            if i == 0 :
                COMBINED.append("WAIT")
            else:
                if df_group.loc[row_index, 'TradingStrategyBollinger'] == "Buy":
                    if df_group.loc[row_index, 'TradingStrategyRSI'] == "positive":
                        if df_group.loc[row_index, 'TradingStrategyOBV'] == "Buy":
                            COMBINED.append("ENTER")
                        else:
                            COMBINED.append("WAIT")
                    else:
                        COMBINED.append("WAIT")
                else:
                    COMBINED.append("WAIT")
        i = i + 1

In [72]:
# convert the series to dataframe with column as COMBINED
dataframe_COMBINED = pd.DataFrame(COMBINED,columns=['COMBINED'])
dataframe_COMBINED

,COMBINED
0,0
1,WAIT
2,WAIT
3,WAIT
4,WAIT
...,...
244897,WAIT
244898,WAIT
244899,WAIT
244900,WAIT


In [73]:
# remove the first row since the append was started with COMBINED[0] - this way this would align to the length of the original dataframe
dataframe_COMBINED_corrected = dataframe_COMBINED[1:]
dataframe_COMBINED_corrected.tail()

,COMBINED
244897,WAIT
244898,WAIT
244899,WAIT
244900,WAIT
244901,WAIT


In [74]:
# add the Combined Strategy value to the existing daily_data dataframe
daily_data['ENTRY'] = dataframe_COMBINED_corrected['COMBINED'].values

In [171]:
# store the data in an excel
daily_data.to_csv('D:\Personal\Analytics course\IPBA\Case Study\Output\Calculated KPIs.csv')

In [76]:
# determine which stocks to buy. first look at price change each day for each stock
daily_data['dailypricechange'] = np.log(daily_data.groupby('Symbol')['Close'].pct_change().add(1))

In [77]:
# store the unique Stock symbols in a separate data frame to calculate the sharpe ratio and rank the stocks
Stock_data = []
stock = daily_data['Symbol'].unique()
series_stock_name = pd.Series(stock)
Stock_data = pd.DataFrame(series_stock_name, columns = ['Stock'])
Stock_data

,Stock
0,ABBOTINDIA
1,ACC
2,ADANIPORTS
3,ADANITRANS
4,AMBUJACEM
...,...
95,UBL
96,ULTRACEMCO
97,UPL
98,WIPRO


In [78]:
# find the starting price and the latest price based on the data
Stock_data['Initial Close Price - 10 yrs'] = daily_data.groupby('Symbol')['Close'].head(1).values
Stock_data['Latest Close Price - 10 yrs'] = daily_data.groupby('Symbol')['Close'].tail(1).values
Stock_data['returns - 10 yrs'] = Stock_data['Latest Close Price - 10 yrs'] / Stock_data['Initial Close Price - 10 yrs'] - 1

In [79]:
# find the volatility of the stock
daily_data['daily_returns'] = daily_data.groupby('Symbol')['Close'].apply(lambda x: np.log(x) - np.log(x.shift()))
Stock_data['volatility - 10 yrs'] = daily_data.groupby('Symbol')['daily_returns'].std().values * np.sqrt(252*10) # 10 years of data

In [80]:
# determine the Sortino ratio or Sharpe ratio for deciding on which stocks to buy. Assuming a repo rate of 0.05 from RBI
Stock_data['sharpe_ratio - 10 yrs'] = (Stock_data['returns - 10 yrs'] - 0.05)/Stock_data['volatility - 10 yrs']

In [81]:
# rank the data and sort by rank
Stock_data['rank - 10 yrs'] = Stock_data['sharpe_ratio - 10 yrs'].rank()
Stock_data_ranked = Stock_data.sort_values("rank - 10 yrs", ascending= False)
Stock_data_ranked.reset_index().head(10)

,index,Stock,Initial Close Price - 10 yrs,Latest Close Price - 10 yrs,returns - 10 yrs,volatility - 10 yrs,sharpe_ratio - 10 yrs,rank - 10 yrs
0,0,ABBOTINDIA,807.00,16384.90,19.303470,0.842815,22.844241,100.0
1,73,PAGEIND,863.25,21422.40,23.815986,1.049650,22.641830,99.0
2,9,BAJAJFINSV,359.45,6053.60,15.841285,1.131502,13.956034,98.0
3,85,SHREECEM,1962.45,20611.00,9.502688,0.917266,10.305283,97.0
4,44,HINDUNILVR,264.70,2095.00,6.914620,0.763605,8.989753,96.0
5,67,NESTLEIND,2494.65,16000.50,5.413926,0.765048,7.011226,95.0
6,14,BERGEPAINT,60.45,590.55,8.769231,1.249094,6.980444,94.0
7,77,PGHH,1719.20,9866.50,4.739007,0.728992,6.432175,93.0
8,78,PIDILITIND,193.75,1463.00,6.550968,1.109405,5.859868,92.0
9,94,TORNTPHARM,390.45,2817.20,6.215264,1.130188,5.455079,91.0


In [82]:
# determine the performing stocks over past 1 year as of 31-Aug-2020

# select the records between 01-Sep-2019 to 31-Aug-2020 - across the symbols
daily_data['Date'] = pd.to_datetime(daily_data['Date'])
start_date = '31-08-2019'
end_date = '31-08-2020'
mask = (daily_data['Date'] > start_date) & (daily_data['Date'] <= end_date)
last_year_data = daily_data.loc[mask].reset_index(drop=True)
last_year_data

# find the starting price and the latest price based on the data
Stock_data['Initial Close Price - 1 yrs'] = last_year_data.groupby('Symbol')['Close'].head(1).values
Stock_data['Latest Close Price - 1 yrs'] = last_year_data.groupby('Symbol')['Close'].tail(1).values
Stock_data['returns - 1 yrs'] = Stock_data['Latest Close Price - 1 yrs'] / Stock_data['Initial Close Price - 1 yrs'] - 1

# find the volatility of the stock
last_year_data['daily_returns'] = last_year_data.groupby('Symbol')['Close'].apply(lambda x: np.log(x) - np.log(x.shift()))
Stock_data['volatility - 1 yrs'] = last_year_data.groupby('Symbol')['daily_returns'].std().values * np.sqrt(252) # 1 years of data

# determine the Sortino ratio or Sharpe ratio for deciding on which stocks to buy. Assuming a repo rate of 0.05 from RBI
Stock_data['sharpe_ratio - 1 yrs'] = (Stock_data['returns - 1 yrs'] - 0.05)/Stock_data['volatility - 1 yrs']

# rank the data and sort by rank
Stock_data['rank - 1 yrs'] = Stock_data['sharpe_ratio - 1 yrs'].rank()
Stock_data_ranked = Stock_data.sort_values("rank - 1 yrs", ascending= False)
Stock_data_ranked.reset_index(drop=True).head(10)

,Stock,Initial Close Price - 10 yrs,Latest Close Price - 10 yrs,returns - 10 yrs,volatility - 10 yrs,sharpe_ratio - 10 yrs,rank - 10 yrs,Initial Close Price - 1 yrs,Latest Close Price - 1 yrs,returns - 1 yrs,volatility - 1 yrs,sharpe_ratio - 1 yrs,rank - 1 yrs
0,DIVISLAB,670.85,3064.90,3.568682,1.169864,3.007770,82.0,1639.45,3122.80,0.904785,0.344000,2.484838,100.0
1,DRREDDY,1142.15,5112.05,3.475813,0.850012,4.030311,89.0,2532.80,4264.70,0.683789,0.323453,1.959445,99.0
2,ABBOTINDIA,807.00,16384.90,19.303470,0.842815,22.844241,100.0,9445.90,16377.10,0.733779,0.366673,1.864817,98.0
3,BIOCON,279.35,452.65,0.620369,1.638559,0.348092,54.0,225.40,376.15,0.668811,0.368908,1.677413,97.0
4,MUTHOOTFIN,175.90,1153.75,5.559125,1.352696,4.072699,90.0,606.50,1146.75,0.890767,0.566504,1.484132,96.0
5,RELIANCE,1075.50,2225.25,1.069038,1.127111,0.904115,66.0,1206.40,2080.70,0.724718,0.456416,1.478295,95.0
6,CADILAHC,663.65,398.25,-0.399910,1.867005,-0.240979,31.0,228.95,371.25,0.621533,0.390087,1.465144,94.0
7,TORNTPHARM,390.45,2817.20,6.215264,1.130188,5.455079,91.0,1713.40,2661.60,0.553403,0.384191,1.310291,93.0
8,DMART,641.60,2184.80,2.405237,1.063935,2.213705,79.0,1508.00,2251.25,0.492871,0.345470,1.281940,92.0
9,CIPLA,337.55,773.00,1.290031,0.831547,1.491234,71.0,469.55,713.55,0.519646,0.374535,1.253947,91.0


In [125]:
# determine the performing stocks over past 1 week as of 31-Aug-2020

# select the records between 01-Sep-2019 to 31-Aug-2020 - across the symbols
#daily_data['Date'] = pd.to_datetime(daily_data['Date'])
start_date = '24-08-2020'
end_date = '31-08-2020'
mask = (daily_data['Date'] >= start_date) & (daily_data['Date'] < end_date)
last_week_data = daily_data.loc[mask].reset_index(drop=True)
last_week_data

# find the starting price and the latest price based on the data
Stock_data['Initial Close Price - 1 wk'] = last_week_data.groupby('Symbol')['Close'].head(1).values
Stock_data['Latest Close Price - 1 wk'] = last_week_data.groupby('Symbol')['Close'].tail(1).values
Stock_data['returns - 1 wk'] = Stock_data['Latest Close Price - 1 wk'] / Stock_data['Initial Close Price - 1 wk'] - 1

# find the volatility of the stock
last_week_data['daily_returns'] = last_week_data.groupby('Symbol')['Close'].apply(lambda x: np.log(x) - np.log(x.shift()))
Stock_data['volatility - 1 wk'] = last_week_data.groupby('Symbol')['daily_returns'].std().values * np.sqrt(252/52) # 1 week

# determine the Sortino ratio or Sharpe ratio for deciding on which stocks to buy. Assuming a repo rate of 0.05 from RBI
Stock_data['sharpe_ratio - 1 wk'] = (Stock_data['returns - 1 wk'] - 0.05)/Stock_data['volatility - 1 wk']

# rank the data and sort by rank
Stock_data['rank - 1 wk'] = Stock_data['sharpe_ratio - 1 wk'].rank()
Stock_data_ranked = Stock_data.sort_values("rank - 1 wk", ascending= False)
Stock_data_ranked.reset_index(drop=True).head(10)

,Stock,Initial Close Price - 10 yrs,Latest Close Price - 10 yrs,returns - 10 yrs,volatility - 10 yrs,sharpe_ratio - 10 yrs,rank - 10 yrs,Initial Close Price - 1 yrs,Latest Close Price - 1 yrs,returns - 1 yrs,...,31-Aug-2020 EXIT,31-Jul-2020 ENTRY,31-Jul-2020 EXIT,Combined sharpe_ratio rank,Initial Close Price - 1 wk,Latest Close Price - 1 wk,returns - 1 wk,volatility - 1 wk,sharpe_ratio - 1 wk,rank - 1 wk
0,INDUSINDBK,142.85,592.20,3.145607,1.266625,2.443981,81.0,1343.95,630.20,-0.531084,...,WAIT,WAIT,WAIT,32.50,529.15,665.65,0.257961,0.073137,2.843454,100.0
1,SBIN,2291.20,190.30,-0.916943,2.468445,-0.391721,20.0,268.40,212.00,-0.210134,...,WAIT,WAIT,WAIT,23.00,201.45,224.85,0.116158,0.030096,2.198198,99.0
2,TATAMOTORS,826.45,133.50,-0.838466,2.074286,-0.428324,16.0,112.65,143.20,0.271194,...,WAIT,WAIT,WAIT,46.50,121.20,142.80,0.178218,0.083489,1.535748,98.0
3,AXISBANK,992.10,443.70,-0.552767,1.950091,-0.309097,29.0,645.70,496.75,-0.230680,...,WAIT,WAIT,WAIT,30.50,445.80,509.20,0.142216,0.060257,1.530372,97.0
4,SRTRANSFIN,476.70,658.70,0.381791,1.350097,0.245754,49.0,962.85,702.45,-0.270447,...,WAIT,WAIT,WAIT,33.50,684.30,772.75,0.129256,0.075040,1.056192,96.0
5,BANDHANBNK,476.85,285.75,-0.400755,2.002685,-0.225075,32.0,454.55,305.80,-0.327247,...,WAIT,WAIT,WAIT,26.50,294.95,320.05,0.085099,0.044905,0.781624,95.0
6,ICICIBANK,879.70,369.20,-0.580311,1.896861,-0.332292,26.0,392.15,394.60,0.006248,...,WAIT,WAIT,WAIT,48.75,380.35,409.70,0.077166,0.037751,0.719596,94.0
7,NMDC,427.35,82.60,-0.806716,1.139944,-0.751542,3.0,80.50,97.15,0.206832,...,WAIT,WAIT,WAIT,50.25,95.70,107.50,0.123302,0.125608,0.583578,93.0
8,DLF,364.60,155.90,-0.572408,1.523779,-0.408464,18.0,163.55,158.35,-0.031795,...,WAIT,WAIT,WAIT,42.00,157.70,173.05,0.097337,0.111327,0.425204,92.0
9,BANKBARODA,515.70,41.95,-0.918654,2.020957,-0.479305,13.0,91.45,48.95,-0.464735,...,WAIT,WAIT,WAIT,4.75,48.35,51.60,0.067218,0.062170,0.276952,91.0


In [83]:
# determine the performing stocks over past 5 years as of 31-Aug-2020

# select the records between 01-Sep-2015 to 31-Aug-2020 - across the symbols
#daily_data['Date'] = pd.to_datetime(daily_data['Date'])
start_date = '31-08-2015'
end_date = '31-08-2020'
mask = (daily_data['Date'] > start_date) & (daily_data['Date'] <= end_date)
last_5year_data = daily_data.loc[mask].reset_index(drop=True)

# find the starting price and the latest price based on the data
Stock_data['Initial Close Price - 5 yrs'] = last_5year_data.groupby('Symbol')['Close'].head(1).values
Stock_data['Latest Close Price - 5 yrs'] = last_5year_data.groupby('Symbol')['Close'].tail(1).values
Stock_data['returns - 5 yrs'] = Stock_data['Latest Close Price - 5 yrs'] / Stock_data['Initial Close Price - 5 yrs'] - 1

# find the volatility of the stock
last_5year_data['daily_returns'] = last_5year_data.groupby('Symbol')['Close'].apply(lambda x: np.log(x) - np.log(x.shift()))
Stock_data['volatility - 5 yrs'] = last_5year_data.groupby('Symbol')['daily_returns'].std().values * np.sqrt(252*5) # 1 years of data

# determine the Sortino ratio or Sharpe ratio for deciding on which stocks to buy. Assuming a repo rate of 0.05 from RBI
Stock_data['sharpe_ratio - 5 yrs'] = (Stock_data['returns - 5 yrs'] - 0.05)/Stock_data['volatility - 5 yrs']

# rank the data and sort by rank
Stock_data['rank - 5 yrs'] = Stock_data['sharpe_ratio - 5 yrs'].rank()
Stock_data_ranked = Stock_data.sort_values("rank - 5 yrs", ascending= False)
Stock_data_ranked.reset_index(drop=True).head(10)

,Stock,Initial Close Price - 10 yrs,Latest Close Price - 10 yrs,returns - 10 yrs,volatility - 10 yrs,sharpe_ratio - 10 yrs,rank - 10 yrs,Initial Close Price - 1 yrs,Latest Close Price - 1 yrs,returns - 1 yrs,volatility - 1 yrs,sharpe_ratio - 1 yrs,rank - 1 yrs,Initial Close Price - 5 yrs,Latest Close Price - 5 yrs,returns - 5 yrs,volatility - 5 yrs,sharpe_ratio - 5 yrs,rank - 5 yrs
0,ADANITRANS,39.95,247.20,5.187735,1.606240,3.198610,84.0,234.85,253.30,0.078561,0.536622,0.053223,61.0,29.10,253.30,7.704467,1.132300,6.760103,100.0
1,MUTHOOTFIN,175.90,1153.75,5.559125,1.352696,4.072699,90.0,606.50,1146.75,0.890767,0.566504,1.484132,96.0,173.30,1146.75,5.617138,0.929094,5.992009,99.0
2,ABBOTINDIA,807.00,16384.90,19.303470,0.842815,22.844241,100.0,9445.90,16377.10,0.733779,0.366673,1.864817,98.0,4712.25,16377.10,2.475431,0.605319,4.006863,98.0
3,NAUKRI,874.80,3609.15,3.125686,1.436763,2.140705,77.0,2036.50,3263.65,0.602578,0.513472,1.076160,88.0,749.55,3263.65,3.354146,0.863604,3.825994,97.0
4,DMART,641.60,2184.80,2.405237,1.063935,2.213705,79.0,1508.00,2251.25,0.492871,0.345470,1.281940,92.0,641.60,2251.25,2.508806,0.756997,3.248105,96.0
5,BAJAJFINSV,359.45,6053.60,15.841285,1.131502,13.956034,98.0,7026.30,6190.35,-0.118974,0.553359,-0.305361,35.0,1772.60,6190.35,2.492243,0.821270,2.973739,95.0
6,TITAN,1421.50,1198.65,-0.156771,3.049923,-0.067795,42.0,1059.65,1100.50,0.038550,0.423771,-0.027018,53.0,334.95,1100.50,2.285565,0.752673,2.970166,94.0
7,NESTLEIND,2494.65,16000.50,5.413926,0.765048,7.011226,95.0,12690.95,15949.55,0.256766,0.326688,0.632915,86.0,6003.60,15949.55,1.656664,0.551012,2.915842,93.0
8,HINDUNILVR,264.70,2095.00,6.914620,0.763605,8.989753,96.0,1841.85,2117.35,0.149578,0.343353,0.290016,77.0,853.75,2117.35,1.480059,0.517912,2.761200,92.0
9,PIDILITIND,193.75,1463.00,6.550968,1.109405,5.859868,92.0,1348.65,1402.80,0.040151,0.357004,-0.027587,52.0,553.45,1402.80,1.534646,0.585503,2.535678,91.0


In [140]:
# optimize the ranked stocks based on years
# assumption - take 70% weightage for past 1 year, 20% for past 5 years and 10% for past 10 years
Stock_data['Combined sharpe_ratio rank'] = (Stock_data['rank - 1 yrs'] * 0.7) + (Stock_data['rank - 5 yrs'] * 0.2) + (Stock_data['rank - 10 yrs'] * 0.1)
# sort by rank
Stock_data_ranked = Stock_data.sort_values("Combined sharpe_ratio rank", ascending= False)
Stock_data_ranked.reset_index(drop=True).head(10)
Stock_data_ranked['31-Aug-2020 ENTRY'].head(50)
type(Stock_data_ranked)

pandas.core.frame.DataFrame

In [173]:
# Store the selected stocks into a dataframe
# Pick top 10 stocks based on combined rank order and where entry signal (to buy) as on 31-Aug-2020 is ENTER
Stock_data_Selected = []
i = 0
j = 0
for row_index, row in Stock_data_ranked.iterrows():
    if j < 10:
#        if Stock_data_ranked.loc[row_index, 'Combined sharpe_ratio rank'] >= 90:
        if Stock_data_ranked.loc[row_index, '31-Aug-2020 ENTRY'] == "ENTER":
            Stock_data_Selected.append(row['Stock'])
            j = j + 1
    i = i + 1
Stock_data_Selected
    
plan_next_day = "No"
if(len(Stock_data_Selected) == 0):
    print ("NO STOCKS to BUY")
    plan_next_day = "Yes"

In [ ]:
# determine the day (Mon-Fri) of the day so that we do a weekly transaction, rather than on a day basis - else the cost would increase due to tx fees
import datetime as dt
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
daily_data['WeekdayName']=daily_data['Date'].dt.dayofweek.map(dayOfWeek)
# determine the year, week number and day of transaction - that way we can use this for picking the last working day of each week
daily_data['TransactionYear'] = daily_data['Date'].dt.year
daily_data['TransactionWeek'] = daily_data['Date'].dt.week
daily_data['TransactionDay'] = daily_data['Date'].dt.dayofweek

In [ ]:
daily_data['MaxWeekDay'] = daily_data.groupby(['Symbol','TransactionYear','TransactionWeek'],sort=False)['TransactionDay'].transform(max)
daily_data.tail(50)

In [ ]:
# extract the rows where MaxWeekDay matches to WeekDay
week_data = daily_data[daily_data['TransactionDay'] == daily_data['MaxWeekDay']]

In [ ]:
# extract the rows belonging to last day of each week
transyear = raw_data['TransactionYear'].map(str)
transweek = raw_data['TransactionWeek'].apply(lambda x: '{0:0>2}'.format(x)).map(str)
transday = raw_data['TransactionDay'].map(str)
#raw_data[['TransactionYear','TransactionWeek']].apply(lambda x: "".join(x),axis=1)
transyearweekday = transyear + transweek + transday
type(transyearweekday)
transyearweekday

In [ ]:
AbbotIndia = raw_data[raw_data.Symbol == "ABBOTINDIA"]
print(AbbotIndia)

In [ ]:
plt.plot(AbbotIndia.Date, AbbotIndia['Close'], label='Close')
plt.plot(AbbotIndia.Date, AbbotIndia['SMA-200'], label='SMA-200', color='green')
plt.plot(AbbotIndia.Date, AbbotIndia['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
from datetime import datetime
mydate = AbbotIndia['Date'].dt.year
print(mydate)
AbbotIndia[mydate == 2020]

In [ ]:
leadingchar = "Stock_"
trailingchar = ".xlsx"
stocklist = np.char.strip(filepaths,leadingchar)
stocklist = np.char.strip(stocklist,trailingchar)
print(stocklist)

In [ ]:
#for i in stocklist:
#    print(i)
#    raw_data[raw_data.Symbol == stocklist[i]]
#stock(i) = raw_data[raw_data.Symbol == stocklist[1]]
AbbotIndia = raw_data[raw_data.Symbol == "ABBOTINDIA"]
plt.plot(AbbotIndia.Date, AbbotIndia['Close'], label='Close')
plt.plot(AbbotIndia.Date, AbbotIndia['SMA-200'], label='SMA-200', color='green')
plt.plot(AbbotIndia.Date, AbbotIndia['EMA-200'], label='EMA-200', color='red')
plt.plot(AbbotIndia.Date, AbbotIndia['rsi'], label='RSI', color='orange')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Acc = raw_data[raw_data.Symbol == "ACC"]
plt.plot(Acc.Date, Acc['Close'], label='Close')
plt.plot(Acc.Date, Acc['SMA-200'], label='SMA-200', color='green')
plt.plot(Acc.Date, Acc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Adaniports = raw_data[raw_data.Symbol == "ADANIPORTS"]
plt.plot(Adaniports.Date, Adaniports['Close'], label='Close')
plt.plot(Adaniports.Date, Adaniports['SMA-200'], label='SMA-200', color='green')
plt.plot(Adaniports.Date, Adaniports['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Adanitrans = raw_data[raw_data.Symbol == "ADANITRANS"]
plt.plot(Adanitrans.Date, Adanitrans['Close'], label='Close')
plt.plot(Adanitrans.Date, Adanitrans['SMA-200'], label='SMA-200', color='green')
plt.plot(Adanitrans.Date, Adanitrans['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Ambujacem = raw_data[raw_data.Symbol == "AMBUJACEM"]
plt.plot(Ambujacem.Date, Ambujacem['Close'], label='Close')
plt.plot(Ambujacem.Date, Ambujacem['SMA-200'], label='SMA-200', color='green')
plt.plot(Ambujacem.Date, Ambujacem['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Asianpaint = raw_data[raw_data.Symbol == "ASIANPAINT"]
plt.plot(Asianpaint.Date, Asianpaint['Close'], label='Close')
plt.plot(Asianpaint.Date, Asianpaint['SMA-200'], label='SMA-200', color='green')
plt.plot(Asianpaint.Date, Asianpaint['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Auropharma = raw_data[raw_data.Symbol == "AUROPHARMA"]
plt.plot(Auropharma.Date, Auropharma['Close'], label='Close')
plt.plot(Auropharma.Date, Auropharma['SMA-200'], label='SMA-200', color='green')
plt.plot(Auropharma.Date, Auropharma['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Axisbank = raw_data[raw_data.Symbol == "AXISBANK"]
plt.plot(Axisbank.Date, Axisbank['Close'], label='Close')
plt.plot(Axisbank.Date, Axisbank['SMA-200'], label='SMA-200', color='green')
plt.plot(Axisbank.Date, Axisbank['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BajajAuto = raw_data[raw_data.Symbol == "BAJAJ-AUTO"]
plt.plot(BajajAuto.Date, BajajAuto['Close'], label='Close')
plt.plot(BajajAuto.Date, BajajAuto['SMA-200'], label='SMA-200', color='green')
plt.plot(BajajAuto.Date, BajajAuto['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Bajajfinsv = raw_data[raw_data.Symbol == "BAJAJFINSV"]
plt.plot(Bajajfinsv.Date, Bajajfinsv['Close'], label='Close')
plt.plot(Bajajfinsv.Date, Bajajfinsv['SMA-200'], label='SMA-200', color='green')
plt.plot(Bajajfinsv.Date, Bajajfinsv['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Bajajhldng = raw_data[raw_data.Symbol == "BAJAJHLDNG"]
plt.plot(Bajajhldng.Date, Bajajhldng['Close'], label='Close')
plt.plot(Bajajhldng.Date, Bajajhldng['SMA-200'], label='SMA-200', color='green')
plt.plot(Bajajhldng.Date, Bajajhldng['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BajFinance = raw_data[raw_data.Symbol == "BAJFINANCE"]
plt.plot(BajFinance.Date, BajFinance['Close'], label='Close')
plt.plot(BajFinance.Date, BajFinance['SMA-200'], label='SMA-200', color='green')
plt.plot(BajFinance.Date, BajFinance['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BandhanBnk = raw_data[raw_data.Symbol == "BANDHANBNK"]
plt.plot(BandhanBnk.Date, BandhanBnk['Close'], label='Close')
plt.plot(BandhanBnk.Date, BandhanBnk['SMA-200'], label='SMA-200', color='green')
plt.plot(BandhanBnk.Date, BandhanBnk['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BankBaroda = raw_data[raw_data.Symbol == "BANKBARODA"]
plt.plot(BankBaroda.Date, BankBaroda['Close'], label='Close')
plt.plot(BankBaroda.Date, BankBaroda['SMA-200'], label='SMA-200', color='green')
plt.plot(BankBaroda.Date, BankBaroda['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BergePaint = raw_data[raw_data.Symbol == "BERGEPAINT"]
plt.plot(BergePaint.Date, BergePaint['Close'], label='Close')
plt.plot(BergePaint.Date, BergePaint['SMA-200'], label='SMA-200', color='green')
plt.plot(BergePaint.Date, BergePaint['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BhartiArtl = raw_data[raw_data.Symbol == "BHARTIARTL"]
plt.plot(BhartiArtl.Date, BhartiArtl['Close'], label='Close')
plt.plot(BhartiArtl.Date, BhartiArtl['SMA-200'], label='SMA-200', color='green')
plt.plot(BhartiArtl.Date, BhartiArtl['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Biocon = raw_data[raw_data.Symbol == "BIOCON"]
plt.plot(Biocon.Date, Biocon['Close'], label='Close')
plt.plot(Biocon.Date, Biocon['SMA-200'], label='SMA-200', color='green')
plt.plot(Biocon.Date, Biocon['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
BoschLtd = raw_data[raw_data.Symbol == "BOSCHLTD"]
plt.plot(BoschLtd.Date, BoschLtd['Close'], label='Close')
plt.plot(BoschLtd.Date, BoschLtd['SMA-200'], label='SMA-200', color='green')
plt.plot(BoschLtd.Date, BoschLtd['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Bpcl = raw_data[raw_data.Symbol == "BPCL"]
plt.plot(Bpcl.Date, Bpcl['Close'], label='Close')
plt.plot(Bpcl.Date, Bpcl['SMA-200'], label='SMA-200', color='green')
plt.plot(Bpcl.Date, Bpcl['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Britannia = raw_data[raw_data.Symbol == "BRITANNIA"]
plt.plot(Britannia.Date, Britannia['Close'], label='Close')
plt.plot(Britannia.Date, Britannia['SMA-200'], label='SMA-200', color='green')
plt.plot(Britannia.Date, Britannia['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Cadilahc = raw_data[raw_data.Symbol == "CADILAHC"]
plt.plot(Cadilahc.Date, Cadilahc['Close'], label='Close')
plt.plot(Cadilahc.Date, Cadilahc['SMA-200'], label='SMA-200', color='green')
plt.plot(Cadilahc.Date, Cadilahc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Cipla = raw_data[raw_data.Symbol == "CIPLA"]
plt.plot(Cipla.Date, Cipla['Close'], label='Close')
plt.plot(Cipla.Date, Cipla['SMA-200'], label='SMA-200', color='green')
plt.plot(Cipla.Date, Cipla['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
CoalIndia = raw_data[raw_data.Symbol == "COALINDIA"]
plt.plot(CoalIndia.Date, CoalIndia['Close'], label='Close')
plt.plot(CoalIndia.Date, CoalIndia['SMA-200'], label='SMA-200', color='green')
plt.plot(CoalIndia.Date, CoalIndia['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Colpal = raw_data[raw_data.Symbol == "COLPAL"]
plt.plot(Colpal.Date, Colpal['Close'], label='Close')
plt.plot(Colpal.Date, Colpal['SMA-200'], label='SMA-200', color='green')
plt.plot(Colpal.Date, Colpal['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Concor = raw_data[raw_data.Symbol == "CONCOR"]
plt.plot(Concor.Date, Concor['Close'], label='Close')
plt.plot(Concor.Date, Concor['SMA-200'], label='SMA-200', color='green')
plt.plot(Concor.Date, Concor['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Dabur = raw_data[raw_data.Symbol == "DABUR"]
plt.plot(Dabur.Date, Dabur['Close'], label='Close')
plt.plot(Dabur.Date, Dabur['SMA-200'], label='SMA-200', color='green')
plt.plot(Dabur.Date, Dabur['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
DivisLab = raw_data[raw_data.Symbol == "DIVISLAB"]
plt.plot(DivisLab.Date, DivisLab['Close'], label='Close')
plt.plot(DivisLab.Date, DivisLab['SMA-200'], label='SMA-200', color='green')
plt.plot(DivisLab.Date, DivisLab['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Dlf = raw_data[raw_data.Symbol == "DLF"]
plt.plot(Dlf.Date, Dlf['Close'], label='Close')
plt.plot(Dlf.Date, Dlf['SMA-200'], label='SMA-200', color='green')
plt.plot(Dlf.Date, Dlf['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Dmart = raw_data[raw_data.Symbol == "DMART"]
plt.plot(Dmart.Date, Dmart['Close'], label='Close')
plt.plot(Dmart.Date, Dmart['SMA-200'], label='SMA-200', color='green')
plt.plot(Dmart.Date, Dmart['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
DrReddy = raw_data[raw_data.Symbol == "DRREDDY"]
plt.plot(DrReddy.Date, DrReddy['Close'], label='Close')
plt.plot(DrReddy.Date, DrReddy['SMA-200'], label='SMA-200', color='green')
plt.plot(DrReddy.Date, DrReddy['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
EicherMot = raw_data[raw_data.Symbol == "EICHERMOT"]
plt.plot(EicherMot.Date, EicherMot['Close'], label='Close')
plt.plot(EicherMot.Date, EicherMot['SMA-200'], label='SMA-200', color='green')
plt.plot(EicherMot.Date, EicherMot['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Gail = raw_data[raw_data.Symbol == "GAIL"]
plt.plot(Gail.Date, Gail['Close'], label='Close')
plt.plot(Gail.Date, Gail['SMA-200'], label='SMA-200', color='green')
plt.plot(Gail.Date, Gail['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Gicre = raw_data[raw_data.Symbol == "GICRE"]
plt.plot(Gicre.Date, Gicre['Close'], label='Close')
plt.plot(Gicre.Date, Gicre['SMA-200'], label='SMA-200', color='green')
plt.plot(Gicre.Date, Gicre['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
GodrejCp = raw_data[raw_data.Symbol == "GODREJCP"]
plt.plot(GodrejCp.Date, GodrejCp['Close'], label='Close')
plt.plot(GodrejCp.Date, GodrejCp['SMA-200'], label='SMA-200', color='green')
plt.plot(GodrejCp.Date, GodrejCp['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Grasim = raw_data[raw_data.Symbol == "GRASIM"]
plt.plot(Grasim.Date, Grasim['Close'], label='Close')
plt.plot(Grasim.Date, Grasim['SMA-200'], label='SMA-200', color='green')
plt.plot(Grasim.Date, Grasim['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Havells = raw_data[raw_data.Symbol == "HAVELLS"]
plt.plot(Havells.Date, Havells['Close'], label='Close')
plt.plot(Havells.Date, Havells['SMA-200'], label='SMA-200', color='green')
plt.plot(Havells.Date, Havells['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HclTech = raw_data[raw_data.Symbol == "HCLTECH"]
plt.plot(HclTech.Date, HclTech['Close'], label='Close')
plt.plot(HclTech.Date, HclTech['SMA-200'], label='SMA-200', color='green')
plt.plot(HclTech.Date, HclTech['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Hdfc = raw_data[raw_data.Symbol == "HDFC"]
plt.plot(Hdfc.Date, Hdfc['Close'], label='Close')
plt.plot(Hdfc.Date, Hdfc['SMA-200'], label='SMA-200', color='green')
plt.plot(Hdfc.Date, Hdfc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HdfcAmc = raw_data[raw_data.Symbol == "HDFCAMC"]
plt.plot(HdfcAmc.Date, HdfcAmc['Close'], label='Close')
plt.plot(HdfcAmc.Date, HdfcAmc['SMA-200'], label='SMA-200', color='green')
plt.plot(HdfcAmc.Date, HdfcAmc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HdfcBank = raw_data[raw_data.Symbol == "HDFCBANK"]
plt.plot(HdfcBank.Date, HdfcBank['Close'], label='Close')
plt.plot(HdfcBank.Date, HdfcBank['SMA-200'], label='SMA-200', color='green')
plt.plot(HdfcBank.Date, HdfcBank['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HdfcLife = raw_data[raw_data.Symbol == "HDFCLIFE"]
plt.plot(HdfcLife.Date, HdfcLife['Close'], label='Close')
plt.plot(HdfcLife.Date, HdfcLife['SMA-200'], label='SMA-200', color='green')
plt.plot(HdfcLife.Date, HdfcLife['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HeroMotoCo = raw_data[raw_data.Symbol == "HEROMOTOCO"]
plt.plot(HeroMotoCo.Date, HeroMotoCo['Close'], label='Close')
plt.plot(HeroMotoCo.Date, HeroMotoCo['SMA-200'], label='SMA-200', color='green')
plt.plot(HeroMotoCo.Date, HeroMotoCo['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HindalCo = raw_data[raw_data.Symbol == "HINDALCO"]
plt.plot(HindalCo.Date, HindalCo['Close'], label='Close')
plt.plot(HindalCo.Date, HindalCo['SMA-200'], label='SMA-200', color='green')
plt.plot(HindalCo.Date, HindalCo['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HindPetro = raw_data[raw_data.Symbol == "HINDPETRO"]
plt.plot(HindPetro.Date, HindPetro['Close'], label='Close')
plt.plot(HindPetro.Date, HindPetro['SMA-200'], label='SMA-200', color='green')
plt.plot(HindPetro.Date, HindPetro['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HindUnilvr = raw_data[raw_data.Symbol == "HINDUNILVR"]
plt.plot(HindUnilvr.Date, HindUnilvr['Close'], label='Close')
plt.plot(HindUnilvr.Date, HindUnilvr['SMA-200'], label='SMA-200', color='green')
plt.plot(HindUnilvr.Date, HindUnilvr['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
HindZinc = raw_data[raw_data.Symbol == "HINDZINC"]
plt.plot(HindZinc.Date, HindZinc['Close'], label='Close')
plt.plot(HindZinc.Date, HindZinc['SMA-200'], label='SMA-200', color='green')
plt.plot(HindZinc.Date, HindZinc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
IciciBank = raw_data[raw_data.Symbol == "ICICIBANK"]
plt.plot(IciciBank.Date, IciciBank['Close'], label='Close')
plt.plot(IciciBank.Date, IciciBank['SMA-200'], label='SMA-200', color='green')
plt.plot(IciciBank.Date, IciciBank['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
IciciGi = raw_data[raw_data.Symbol == "ICICIGI"]
plt.plot(IciciGi.Date, IciciGi['Close'], label='Close')
plt.plot(IciciGi.Date, IciciGi['SMA-200'], label='SMA-200', color='green')
plt.plot(IciciGi.Date, IciciGi['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
IciciPruLi = raw_data[raw_data.Symbol == "ICICIPRULI"]
plt.plot(IciciPruLi.Date, IciciPruLi['Close'], label='Close')
plt.plot(IciciPruLi.Date, IciciPruLi['SMA-200'], label='SMA-200', color='green')
plt.plot(IciciPruLi.Date, IciciPruLi['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Igl = raw_data[raw_data.Symbol == "IGL"]
plt.plot(Igl.Date, Igl['Close'], label='Close')
plt.plot(Igl.Date, Igl['SMA-200'], label='SMA-200', color='green')
plt.plot(Igl.Date, Igl['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Indigo = raw_data[raw_data.Symbol == "INDIGO"]
plt.plot(Indigo.Date, Indigo['Close'], label='Close')
plt.plot(Indigo.Date, Indigo['SMA-200'], label='SMA-200', color='green')
plt.plot(Indigo.Date, Indigo['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
IndusIndBk = raw_data[raw_data.Symbol == "INDUSINDBK"]
plt.plot(IndusIndBk.Date, IndusIndBk['Close'], label='Close')
plt.plot(IndusIndBk.Date, IndusIndBk['SMA-200'], label='SMA-200', color='green')
plt.plot(IndusIndBk.Date, IndusIndBk['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
InfraTel = raw_data[raw_data.Symbol == "INFRATEL"]
plt.plot(InfraTel.Date, InfraTel['Close'], label='Close')
plt.plot(InfraTel.Date, InfraTel['SMA-200'], label='SMA-200', color='green')
plt.plot(InfraTel.Date, InfraTel['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Infy = raw_data[raw_data.Symbol == "INFY"]
plt.plot(Infy.Date, Infy['Close'], label='Close')
plt.plot(Infy.Date, Infy['SMA-200'], label='SMA-200', color='green')
plt.plot(Infy.Date, Infy['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Ioc = raw_data[raw_data.Symbol == "IOC"]
plt.plot(Ioc.Date, Ioc['Close'], label='Close')
plt.plot(Ioc.Date, Ioc['SMA-200'], label='SMA-200', color='green')
plt.plot(Ioc.Date, Ioc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Itc = raw_data[raw_data.Symbol == "ITC"]
plt.plot(Itc.Date, Itc['Close'], label='Close')
plt.plot(Itc.Date, Itc['SMA-200'], label='SMA-200', color='green')
plt.plot(Itc.Date, Itc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
JswSteel = raw_data[raw_data.Symbol == "JSWSTEEL"]
plt.plot(JswSteel.Date, JswSteel['Close'], label='Close')
plt.plot(JswSteel.Date, JswSteel['SMA-200'], label='SMA-200', color='green')
plt.plot(JswSteel.Date, JswSteel['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
KotakBank = raw_data[raw_data.Symbol == "KOTAKBANK"]
plt.plot(KotakBank.Date, KotakBank['Close'], label='Close')
plt.plot(KotakBank.Date, KotakBank['SMA-200'], label='SMA-200', color='green')
plt.plot(KotakBank.Date, KotakBank['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Lt = raw_data[raw_data.Symbol == "LT"]
plt.plot(Lt.Date, Lt['Close'], label='Close')
plt.plot(Lt.Date, Lt['SMA-200'], label='SMA-200', color='green')
plt.plot(Lt.Date, Lt['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Lupin = raw_data[raw_data.Symbol == "LUPIN"]
plt.plot(Lupin.Date, Lupin['Close'], label='Close')
plt.plot(Lupin.Date, Lupin['SMA-200'], label='SMA-200', color='green')
plt.plot(Lupin.Date, Lupin['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
MM = raw_data[raw_data.Symbol == "M&M"]
plt.plot(MM.Date, MM['Close'], label='Close')
plt.plot(MM.Date, MM['SMA-200'], label='SMA-200', color='green')
plt.plot(MM.Date, MM['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Marico = raw_data[raw_data.Symbol == "MARICO"]
plt.plot(Marico.Date, Marico['Close'], label='Close')
plt.plot(Marico.Date, Marico['SMA-200'], label='SMA-200', color='green')
plt.plot(Marico.Date, Marico['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Maruti = raw_data[raw_data.Symbol == "MARUTI"]
plt.plot(Maruti.Date, Maruti['Close'], label='Close')
plt.plot(Maruti.Date, Maruti['SMA-200'], label='SMA-200', color='green')
plt.plot(Maruti.Date, Maruti['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
McDowellN = raw_data[raw_data.Symbol == "MCDOWELL-N"]
plt.plot(McDowellN.Date, McDowellN['Close'], label='Close')
plt.plot(McDowellN.Date, McDowellN['SMA-200'], label='SMA-200', color='green')
plt.plot(McDowellN.Date, McDowellN['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
MotherSumi = raw_data[raw_data.Symbol == "MOTHERSUMI"]
plt.plot(MotherSumi.Date, MotherSumi['Close'], label='Close')
plt.plot(MotherSumi.Date, MotherSumi['SMA-200'], label='SMA-200', color='green')
plt.plot(MotherSumi.Date, MotherSumi['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
MuthootFin = raw_data[raw_data.Symbol == "MUTHOOTFIN"]
plt.plot(MuthootFin.Date, MuthootFin['Close'], label='Close')
plt.plot(MuthootFin.Date, MuthootFin['SMA-200'], label='SMA-200', color='green')
plt.plot(MuthootFin.Date, MuthootFin['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Naukri = raw_data[raw_data.Symbol == "NAUKRI"]
plt.plot(Naukri.Date, Naukri['Close'], label='Close')
plt.plot(Naukri.Date, Naukri['SMA-200'], label='SMA-200', color='green')
plt.plot(Naukri.Date, Naukri['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
NestleInd = raw_data[raw_data.Symbol == "NESTLEIND"]
plt.plot(NestleInd.Date, NestleInd['Close'], label='Close')
plt.plot(NestleInd.Date, NestleInd['SMA-200'], label='SMA-200', color='green')
plt.plot(NestleInd.Date, NestleInd['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Nhpc = raw_data[raw_data.Symbol == "NHPC"]
plt.plot(Nhpc.Date, Nhpc['Close'], label='Close')
plt.plot(Nhpc.Date, Nhpc['SMA-200'], label='SMA-200', color='green')
plt.plot(Nhpc.Date, Nhpc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Nmdc = raw_data[raw_data.Symbol == "NMDC"]
plt.plot(Nmdc.Date, Nmdc['Close'], label='Close')
plt.plot(Nmdc.Date, Nmdc['SMA-200'], label='SMA-200', color='green')
plt.plot(Nmdc.Date, Nmdc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Ntpc = raw_data[raw_data.Symbol == "NTPC"]
plt.plot(Ntpc.Date, Ntpc['Close'], label='Close')
plt.plot(Ntpc.Date, Ntpc['SMA-200'], label='SMA-200', color='green')
plt.plot(Ntpc.Date, Ntpc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Ofss = raw_data[raw_data.Symbol == "OFSS"]
plt.plot(Ofss.Date, Ofss['Close'], label='Close')
plt.plot(Ofss.Date, Ofss['SMA-200'], label='SMA-200', color='green')
plt.plot(Ofss.Date, Ofss['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Ongc = raw_data[raw_data.Symbol == "ONGC"]
plt.plot(Ongc.Date, Ongc['Close'], label='Close')
plt.plot(Ongc.Date, Ongc['SMA-200'], label='SMA-200', color='green')
plt.plot(Ongc.Date, Ongc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
PageInd = raw_data[raw_data.Symbol == "PAGEIND"]
plt.plot(PageInd.Date, PageInd['Close'], label='Close')
plt.plot(PageInd.Date, PageInd['SMA-200'], label='SMA-200', color='green')
plt.plot(PageInd.Date, PageInd['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Pel = raw_data[raw_data.Symbol == "PEL"]
plt.plot(Pel.Date, Pel['Close'], label='Close')
plt.plot(Pel.Date, Pel['SMA-200'], label='SMA-200', color='green')
plt.plot(Pel.Date, Pel['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
PetroNet = raw_data[raw_data.Symbol == "PETRONET"]
plt.plot(PetroNet.Date, PetroNet['Close'], label='Close')
plt.plot(PetroNet.Date, PetroNet['SMA-200'], label='SMA-200', color='green')
plt.plot(PetroNet.Date, PetroNet['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Pfc = raw_data[raw_data.Symbol == "PFC"]
plt.plot(Pfc.Date, Pfc['Close'], label='Close')
plt.plot(Pfc.Date, Pfc['SMA-200'], label='SMA-200', color='green')
plt.plot(Pfc.Date, Pfc['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Pghh = raw_data[raw_data.Symbol == "PGHH"]
plt.plot(Pghh.Date, Pghh['Close'], label='Close')
plt.plot(Pghh.Date, Pghh['SMA-200'], label='SMA-200', color='green')
plt.plot(Pghh.Date, Pghh['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
PidiLitInd = raw_data[raw_data.Symbol == "PIDILITIND"]
plt.plot(PidiLitInd.Date, PidiLitInd['Close'], label='Close')
plt.plot(PidiLitInd.Date, PidiLitInd['SMA-200'], label='SMA-200', color='green')
plt.plot(PidiLitInd.Date, PidiLitInd['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Pnb = raw_data[raw_data.Symbol == "PNB"]
plt.plot(Pnb.Date, Pnb['Close'], label='Close')
plt.plot(Pnb.Date, Pnb['SMA-200'], label='SMA-200', color='green')
plt.plot(Pnb.Date, Pnb['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
PowerGrid = raw_data[raw_data.Symbol == "POWERGRID"]
plt.plot(PowerGrid.Date, PowerGrid['Close'], label='Close')
plt.plot(PowerGrid.Date, PowerGrid['SMA-200'], label='SMA-200', color='green')
plt.plot(PowerGrid.Date, PowerGrid['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Reliance = raw_data[raw_data.Symbol == "RELIANCE"]
plt.plot(Reliance.Date, Reliance['Close'], label='Close')
plt.plot(Reliance.Date, Reliance['SMA-200'], label='SMA-200', color='green')
plt.plot(Reliance.Date, Reliance['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Sbicard = raw_data[raw_data.Symbol == "SBICARD"]
plt.plot(Sbicard.Date, Sbicard['Close'], label='Close')
plt.plot(Sbicard.Date, Sbicard['SMA-200'], label='SMA-200', color='green')
plt.plot(Sbicard.Date, Sbicard['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
SbiLife = raw_data[raw_data.Symbol == "SBILIFE"]
plt.plot(SbiLife.Date, SbiLife['Close'], label='Close')
plt.plot(SbiLife.Date, SbiLife['SMA-200'], label='SMA-200', color='green')
plt.plot(SbiLife.Date, SbiLife['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Sbin = raw_data[raw_data.Symbol == "SBIN"]
plt.plot(Sbin.Date, Sbin['Close'], label='Close')
plt.plot(Sbin.Date, Sbin['SMA-200'], label='SMA-200', color='green')
plt.plot(Sbin.Date, Sbin['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
ShreeCem = raw_data[raw_data.Symbol == "SHREECEM"]
plt.plot(ShreeCem.Date, ShreeCem['Close'], label='Close')
plt.plot(ShreeCem.Date, ShreeCem['SMA-200'], label='SMA-200', color='green')
plt.plot(ShreeCem.Date, ShreeCem['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Siemens = raw_data[raw_data.Symbol == "SIEMENS"]
plt.plot(Siemens.Date, Siemens['Close'], label='Close')
plt.plot(Siemens.Date, Siemens['SMA-200'], label='SMA-200', color='green')
plt.plot(Siemens.Date, Siemens['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
SRTransFin = raw_data[raw_data.Symbol == "SRTRANSFIN"]
plt.plot(SRTransFin.Date, SRTransFin['Close'], label='Close')
plt.plot(SRTransFin.Date, SRTransFin['SMA-200'], label='SMA-200', color='green')
plt.plot(SRTransFin.Date, SRTransFin['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
SunPharma = raw_data[raw_data.Symbol == "SUNPHARMA"]
plt.plot(SunPharma.Date, SunPharma['Close'], label='Close')
plt.plot(SunPharma.Date, SunPharma['SMA-200'], label='SMA-200', color='green')
plt.plot(SunPharma.Date, SunPharma['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
TataMotors = raw_data[raw_data.Symbol == "TATAMOTORS"]
plt.plot(TataMotors.Date, TataMotors['Close'], label='Close')
plt.plot(TataMotors.Date, TataMotors['SMA-200'], label='SMA-200', color='green')
plt.plot(TataMotors.Date, TataMotors['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
TataSteel = raw_data[raw_data.Symbol == "TATASTEEL"]
plt.plot(TataSteel.Date, TataSteel['Close'], label='Close')
plt.plot(TataSteel.Date, TataSteel['SMA-200'], label='SMA-200', color='green')
plt.plot(TataSteel.Date, TataSteel['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Tcs = raw_data[raw_data.Symbol == "TCS"]
plt.plot(Tcs.Date, Tcs['Close'], label='Close')
plt.plot(Tcs.Date, Tcs['SMA-200'], label='SMA-200', color='green')
plt.plot(Tcs.Date, Tcs['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
TechM = raw_data[raw_data.Symbol == "TECHM"]
plt.plot(TechM.Date, TechM['Close'], label='Close')
plt.plot(TechM.Date, TechM['SMA-200'], label='SMA-200', color='green')
plt.plot(TechM.Date, TechM['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Titan = raw_data[raw_data.Symbol == "TITAN"]
plt.plot(Titan.Date, Titan['Close'], label='Close')
plt.plot(Titan.Date, Titan['SMA-200'], label='SMA-200', color='green')
plt.plot(Titan.Date, Titan['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
TorntPharm = raw_data[raw_data.Symbol == "TORNTPHARM"]
plt.plot(TorntPharm.Date, TorntPharm['Close'], label='Close')
plt.plot(TorntPharm.Date, TorntPharm['SMA-200'], label='SMA-200', color='green')
plt.plot(TorntPharm.Date, TorntPharm['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Ubl = raw_data[raw_data.Symbol == "UBL"]
plt.plot(Ubl.Date, Ubl['Close'], label='Close')
plt.plot(Ubl.Date, Ubl['SMA-200'], label='SMA-200', color='green')
plt.plot(Ubl.Date, Ubl['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
UltraCemCo = raw_data[raw_data.Symbol == "ULTRACEMCO"]
plt.plot(UltraCemCo.Date, UltraCemCo['Close'], label='Close')
plt.plot(UltraCemCo.Date, UltraCemCo['SMA-200'], label='SMA-200', color='green')
plt.plot(UltraCemCo.Date, UltraCemCo['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Upl = raw_data[raw_data.Symbol == "UPL"]
plt.plot(Upl.Date, Upl['Close'], label='Close')
plt.plot(Upl.Date, Upl['SMA-200'], label='SMA-200', color='green')
plt.plot(Upl.Date, Upl['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Wipro = raw_data[raw_data.Symbol == "WIPRO"]
plt.plot(Wipro.Date, Wipro['Close'], label='Close')
plt.plot(Wipro.Date, Wipro['SMA-200'], label='SMA-200', color='green')
plt.plot(Wipro.Date, Wipro['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()

In [ ]:
Zeel = raw_data[raw_data.Symbol == "ZEEL"]
plt.plot(Zeel.Date, Zeel['Close'], label='Close')
plt.plot(Zeel.Date, Zeel['SMA-200'], label='SMA-200', color='green')
plt.plot(Zeel.Date, Zeel['EMA-200'], label='EMA-200', color='red')
plt.legend(loc='upper left')
plt.show()